In [22]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [23]:
import jax
import jax.numpy as jnp
import numpy as np
import matplotlib.pyplot as plt
import evosax

import json


In [24]:
import xylo
import xylo.types as t
import xylo.woods
import xylo.cut

import xylo.opt

import xylo.tuning
import xylo.database

In [27]:
tuning = xylo.tuning.yamaha_YX500R
# wood = t.Wood.make_E_nu(E = 29.100e9, nu = 8.20, rho = 1059)
# init_mean = None

In [28]:
options = xylo.opt.Options(num_generations = 100, strategy = evosax.OpenES(popsize = 2000, num_dims = 3), absolute_tolerance = 1e-3)

In [52]:
for num_dims in range(3,10):
  options = xylo.opt.Options(num_generations = 100, strategy = evosax.OpenES(popsize = 2000, num_dims = num_dims), absolute_tolerance = 1e-3)
  init_mean = None

  # for note in range(57, 89):
  for note in [61]:
    db = xylo.database.get()
    match db.get_best_for_dims(note, num_dims):
      case None:
        pass
      case e:
        init_mean = e

    options_ = options._replace(init_mean = init_mean)
    # options_ = options

    bar = db.get_bar(note)
    wood = db.get_wood(note)

    fundamental = tuning.note_to_freq(note)
    partials, weights = tuning.note_to_weights(note)
    print(f"weights {weights}")
    sol = xylo.opt.optimize_geometry(bar, wood, options_, fundamental, jnp.array(partials), jnp.array(weights))
    spline = sol.best_member

    xylo.database.with_db(lambda db: db.set_best_for_dims(note, num_dims, sol.best_member.tolist(), sol.best_fitness.tolist()))

    init_mean = sol.best_member

    sections = xylo.cut.spline(bar, spline)
    swp = xylo.sweep.sweep(wood, bar, sections, t.sweep_default)

    print(note)
    print("---------------")
    print(swp.harmonics / fundamental)
    print("")


weights [1.0, 0.3, 0.1]
iteration 0
[0.67002442 0.82464598 0.4145144 ] 0.004090484
[ 883.81558328 2650.34086539 5101.58493337] [0.99979138 2.99812315 5.77102368] 0.292835
iteration 10
[0.65965353 0.83377661 0.40560952] 0.004085171
[ 883.81558328 2651.24350356 5096.8749868 ] [0.99979138 2.99914423 5.76569569] 0.292835
iteration 20
[0.6650316  0.82922472 0.40986941] 0.003998235
[ 883.97805995 2651.22298905 5098.50867842] [0.99997518 2.99912103 5.76754375] 0.292835
iteration 30
[0.6650316  0.82922472 0.40986941] 0.003998235
[ 883.97805995 2651.22298905 5098.50867842] [0.99997518 2.99912103 5.76754375] 0.292835
iteration 40
[0.6650316  0.82922472 0.40986941] 0.003998235
[ 883.97805995 2651.22298905 5098.50867842] [0.99997518 2.99912103 5.76754375] 0.292835
iteration 50
[0.6650316  0.82922472 0.40986941] 0.003998235
[ 883.97805995 2651.22298905 5098.50867842] [0.99997518 2.99912103 5.76754375] 0.292835
iteration 60
[0.6650316  0.82922472 0.40986941] 0.003998235
[ 883.97805995 2651.22298905 